In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
import wrangle

In [2]:
df = acquire.get_zillow_data()

In [3]:
df

,parcelid,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,fireplacecnt,garagecarcnt,poolcnt,yearbuilt,taxamount,fips,assessmentyear,landtaxvaluedollarcnt,lotsizesquarefeet,calculatedbathnbr,latitude,longitude
0,14297519,4.0,3.5,3100.0,1023282.0,NaN,2.0,NaN,1998.0,11013.72,6059.0,2016.0,537569.0,4506.0,3.5,33634931.0,-117869207.0
1,17052889,2.0,1.0,1465.0,464000.0,1.0,1.0,NaN,1967.0,5672.48,6111.0,2016.0,376000.0,12647.0,1.0,34449266.0,-119281531.0
2,14186244,3.0,2.0,1243.0,564778.0,NaN,2.0,1.0,1962.0,6488.30,6059.0,2016.0,479489.0,8432.0,2.0,33886168.0,-117823170.0
3,12177905,4.0,3.0,2376.0,145143.0,NaN,NaN,1.0,1970.0,1777.51,6037.0,2016.0,36225.0,13038.0,3.0,34245180.0,-118240722.0
4,12095076,4.0,3.0,2962.0,773303.0,NaN,NaN,1.0,1950.0,9516.26,6037.0,2016.0,496619.0,63000.0,3.0,34145202.0,-118179824.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,4.0,2.0,1633.0,346534.0,NaN,NaN,NaN,1962.0,4175.08,6037.0,2016.0,221068.0,4630.0,2.0,33870815.0,-118070858.0
52437,11000655,2.0,2.0,1286.0,354621.0,NaN,NaN,NaN,1940.0,4478.43,6037.0,2016.0,283704.0,47405.0,2.0,34245368.0,-118282383.0
52438,17239384,4.0,2.0,1612.0,67205.0,1.0,2.0,NaN,1964.0,1107.48,6111.0,2016.0,16522.0,12105.0,2.0,34300140.0,-118706327.0
52439,12773139,3.0,1.0,1032.0,49546.0,NaN,NaN,NaN,1954.0,876.43,6037.0,2016.0,16749.0,5074.0,1.0,34040895.0,-118038169.0


In [5]:
df = wrangle.wrangle_zillow(df)

In [59]:
df['fireplacecnt'] = df['fireplacecnt'].replace(['NaN'], '0') 

In [60]:
df.fireplacecnt.dtype 

dtype('float64')

In [31]:
Initial thoughts:
    most interested in emploring locatation
    -what does the number of fire place have to do with this would they be strong correlation to TAS
    Look at that!

KeyError: "None of [Index([(False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, ...)], dtype='object')] are in the [columns]"

In [19]:
df.value.groupby(df.fireplacecnt).mean().round()#fillna(0)

fireplacecnt
1.0     558953.0
2.0     900929.0
3.0    1327960.0
4.0    1986780.0
5.0    3502500.0
Name: value, dtype: float64